# CRYO2ICE Track Alignment Model

## 1. Introduction

This notebook implements a model for identifying valid CRYO2ICE coincident tracks between CryoSat-2 (CS2) and ICESat-2 (IS2) measurements over the Ross and Weddell seas during austral winters (May-October) from 2021 to 2024. The CRYO2ICE mission phase began in July 2020 when the orbit of CryoSat-2 was modified to maximize the number of coincident observations with ICESat-2.

### 1.1 Track Alignment Requirements

We use the following criteria to define valid CRYO2ICE coincident tracks:

1. **Spatial proximity**: CS2 and IS2 ground tracks must be within 10 km of each other
2. **Temporal proximity**: Acquisition times must differ by 3 hours or less
3. **Intersection duration**: The intersection time must exceed 1 minute

These requirements ensure that we identify satellite passes that are observing the same sea ice conditions with minimal changes due to temporal or spatial differences.

In [2]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import h5py
import netCDF4
import os
import glob
from datetime import datetime, timedelta
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from shapely.geometry import Point, LineString
import re
from scipy.spatial import distance
import pyproj
from tqdm import tqdm

# Set random seed for reproducibility
np.random.seed(42)

# Define the base directory
directory = r'D:\phd\data\chap2'

# Define austral winter months (May to October)
winter_months = [5, 6, 7, 8, 9, 10]

## 2. Data Loading

First, we need to load the CryoSat-2 and ICESat-2 data from both the Ross and Weddell seas during austral winters. We'll use the previously analysed datasets and organise them by region and year.

In [3]:
def get_data_paths(base_dir, region, years):
    """
    Get file paths for CS2 and IS2 data for a specific region and years.
    
    Parameters:
    -----------
    base_dir : str
        The base directory where data is stored
    region : str
        'ross' or 'weddell'
    years : list
        List of years to include
        
    Returns:
    --------
    dict: Dictionary with CS2 and IS2 file paths organized by year
    """
    data_paths = {}
    
    for year in years:
        data_paths[year] = {'cs2': [], 'is2': []}
        
        # Get CryoSat-2 files
        if region == 'weddell':
            cs2_dir = os.path.join(base_dir, f'cs2_l2_sar_basel_e_weddell_winter/{year}')
            cs2_files = glob.glob(os.path.join(cs2_dir, '*.nc'))
            data_paths[year]['cs2'] = cs2_files
        elif region == 'ross':
            # For Ross Sea, combine Eastern and Western Hemisphere files
            cs2_dir_eh = os.path.join(base_dir, f'cs2_l2_sar_basel_e_ross_winter/EH/{year}')
            cs2_dir_wh = os.path.join(base_dir, f'cs2_l2_sar_basel_e_ross_winter/WH/{year}')
            
            cs2_files_eh = glob.glob(os.path.join(cs2_dir_eh, '*.nc'))
            cs2_files_wh = glob.glob(os.path.join(cs2_dir_wh, '*.nc'))
            data_paths[year]['cs2'] = cs2_files_eh + cs2_files_wh
        
        # Get ICESat-2 files
        is2_dir = os.path.join(base_dir, f'is2_atl10v6_{region}_winter/{year}')
        is2_files = glob.glob(os.path.join(is2_dir, '*.h5'))
        data_paths[year]['is2'] = is2_files
    
    return data_paths

# Define the years to analyze
years_to_analyze = [2021, 2022, 2023, 2024]

# Get file paths for both regions
weddell_data = get_data_paths(directory, 'weddell', years_to_analyze)
ross_data = get_data_paths(directory, 'ross', years_to_analyze)

# Print summary of available data
for region_name, region_data in [("Weddell", weddell_data), ("Ross", ross_data)]:
    print(f"\n{region_name} Sea data summary:")
    for year, data in region_data.items():
        print(f"  {year}: {len(data['cs2'])} CS2 files, {len(data['is2'])} IS2 files")


Weddell Sea data summary:
  2021: 1285 CS2 files, 1270 IS2 files
  2022: 1280 CS2 files, 1319 IS2 files
  2023: 1403 CS2 files, 1010 IS2 files
  2024: 1402 CS2 files, 939 IS2 files

Ross Sea data summary:
  2021: 1022 CS2 files, 972 IS2 files
  2022: 1011 CS2 files, 987 IS2 files
  2023: 1020 CS2 files, 928 IS2 files
  2024: 1026 CS2 files, 725 IS2 files


## 3. Track Alignment Model Implementation

Now we'll implement the model to identify valid CRYO2ICE coincident tracks based on the specified requirements. This involves:

1. Extracting track information from both satellites (coordinates and timestamps)
2. Calculating spatial distances between tracks
3. Determining temporal differences
4. Estimating intersection times
5. Filtering tracks that meet all criteria

In [4]:
def extract_cs2_track_info(cs2_file):
    """
    Extract track coordinates, timestamps, and freeboard from CryoSat-2 file
    """
    try:
        with netCDF4.Dataset(cs2_file, 'r') as ds:
            # Extract time information
            if 'time_20_ku' in ds.variables:
                time_var = ds.variables['time_20_ku'][:]
                ref_time_str = ds.variables['time_20_ku'].units
                # Parse reference time (usually something like "seconds since 2000-01-01 00:00:00.0")
                ref_time = datetime.strptime(ref_time_str.split("seconds since ")[1].split('.')[0], "%Y-%m-%d %H:%M:%S")
                timestamps = [ref_time + timedelta(seconds=float(t)) for t in time_var]
            else:
                # Extract time from filename if variable not available
                # Example: CS_LTA__SIR_SAR_2__20210501T004324_20210501T004940_E001_segment_640.nc
                filename = os.path.basename(cs2_file)
                time_match = re.search(r'(\d{8}T\d{6})_(\d{8}T\d{6})', filename)
                if time_match:
                    start_time_str = time_match.group(1)
                    end_time_str = time_match.group(2)
                    start_time = datetime.strptime(start_time_str, "%Y%m%dT%H%M%S")
                    end_time = datetime.strptime(end_time_str, "%Y%m%dT%H%M%S")
                    
                    # Create a sequence of timestamps spanning the time range
                    duration = (end_time - start_time).total_seconds()
                    num_points = 100  # Placeholder number of points
                    timestamps = [start_time + timedelta(seconds=i*duration/num_points) for i in range(num_points)]
                else:
                    return None
            
            # Extract coordinates
            lat = ds.variables['lat_poca_20_ku'][:]
            lon = ds.variables['lon_poca_20_ku'][:]
            
            # Extract freeboard
            freeboard = ds.variables['radar_freeboard_20_ku'][:]
            
            # Filter out invalid values
            valid_mask = ~np.isnan(freeboard) & (freeboard > -100) & (freeboard < 100)
            lat = lat[valid_mask]
            lon = lon[valid_mask]
            
            if len(timestamps) > len(lat):
                timestamps = [timestamps[i] for i in range(len(lat))]
            elif len(timestamps) < len(lat):
                timestamps = timestamps + [timestamps[-1]] * (len(lat) - len(timestamps))
            
            return {
                'lat': lat,
                'lon': lon,
                'time': timestamps,
                'source': 'cs2',
                'filename': os.path.basename(cs2_file)
            }
    except Exception as e:
        print(f"Error processing CS2 file {os.path.basename(cs2_file)}: {str(e)}")
        return None

def extract_is2_track_info(is2_file):
    """
    Extract track coordinates, timestamps, and freeboard from ICESat-2 .h5 file
    """
    # Ensure we're only processing .h5 files
    if not is2_file.endswith('.h5'):
        print(f"Skipping non-h5 file: {os.path.basename(is2_file)}")
        return None
        
    try:
        with h5py.File(is2_file, 'r') as f:
            all_lat = []
            all_lon = []
            all_time = []
            
            # Try to extract time from filename first for better efficiency
            # Example: ATL10-02_20210501044315_05711101_006_01.h5
            filename = os.path.basename(is2_file)
            file_time_match = re.search(r'_(\d{8})(\d{6})_', filename)
            file_timestamp = None
            
            if file_time_match:
                date_str = file_time_match.group(1)  # YYYYMMDD
                time_str = file_time_match.group(2)  # HHMMSS
                try:
                    file_timestamp = datetime.strptime(f"{date_str}{time_str}", "%Y%m%d%H%M%S")
                except ValueError:
                    file_timestamp = None
            
            # Process all beams
            beams = ['gt1l', 'gt1r', 'gt2l', 'gt2r', 'gt3l', 'gt3r']
            for beam in beams:
                if beam not in f:
                    continue
                
                # Try to find latitude and longitude
                try:
                    lon = np.array(f[f"{beam}/freeboard_segment/longitude"])
                    lat = np.array(f[f"{beam}/freeboard_segment/latitude"])
                    
                    # Try to get timestamps
                    if 'delta_time' in f[f"{beam}/freeboard_segment"]:
                        delta_time = np.array(f[f"{beam}/freeboard_segment/delta_time"])
                        # ICESat-2 reference time is 2018-01-01 00:00:00
                        ref_time = datetime(2018, 1, 1)
                        timestamps = [ref_time + timedelta(seconds=float(dt)) for dt in delta_time]
                    else:
                        # Use the timestamp from filename if available, otherwise use placeholder
                        if file_timestamp:
                            timestamps = [file_timestamp] * len(lat)
                        else:
                            # Last resort: extract from general filename pattern
                            time_match = re.search(r'_(\d{14})_', filename)
                            if time_match:
                                time_str = time_match.group(1)
                                timestamp = datetime.strptime(time_str, "%Y%m%d%H%M%S")
                                timestamps = [timestamp] * len(lat)
                            else:
                                # Placeholder if no time information found
                                timestamps = [datetime.now()] * len(lat)
                    
                    # Try to get freeboard to filter valid points
                    try:
                        freeboard = np.array(f[f"{beam}/freeboard_segment/beam_fb_height"])
                        valid_idx = ~np.isnan(freeboard) & ~np.isinf(freeboard) & (freeboard < 10) & (freeboard >= 0)
                    except:
                        try:
                            freeboard = np.array(f[f"{beam}/freeboard_segment/heights/height_segment_height"])
                            valid_idx = ~np.isnan(freeboard) & ~np.isinf(freeboard) & (freeboard < 10) & (freeboard > -10)
                        except:
                            valid_idx = np.ones(len(lat), dtype=bool)
                    
                    all_lat.extend(lat[valid_idx])
                    all_lon.extend(lon[valid_idx])
                    all_time.extend([timestamps[i] for i in range(len(valid_idx)) if valid_idx[i]])
                    
                except Exception as e:
                    continue
            
            if not all_lat:
                return None
                
            return {
                'lat': np.array(all_lat),
                'lon': np.array(all_lon),
                'time': all_time,
                'source': 'is2',
                'filename': os.path.basename(is2_file)
            }
    except Exception as e:
        print(f"Error processing IS2 file {os.path.basename(is2_file)}: {str(e)}")
        return None

def haversine_distance(lon1, lat1, lon2, lat2):
    """
    Calculate the great-circle distance between two points on Earth
    using the Haversine formula
    
    Returns distance in kilometers
    """
    # Convert decimal degrees to radians
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])
    
    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    r = 6371  # Radius of earth in kilometers
    
    return c * r

def track_min_distance(track1, track2):
    """
    Calculate minimum distance between two tracks
    
    Returns minimum distance in kilometers and the corresponding points
    """
    # Use a subset of points to speed up calculation for long tracks
    max_points = 1000
    if len(track1['lat']) > max_points:
        idx1 = np.linspace(0, len(track1['lat'])-1, max_points).astype(int)
        lat1 = track1['lat'][idx1]
        lon1 = track1['lon'][idx1]
    else:
        lat1 = track1['lat']
        lon1 = track1['lon']
        
    if len(track2['lat']) > max_points:
        idx2 = np.linspace(0, len(track2['lat'])-1, max_points).astype(int)
        lat2 = track2['lat'][idx2]
        lon2 = track2['lon'][idx2]
    else:
        lat2 = track2['lat']
        lon2 = track2['lon']
    
    # Calculate distance matrix
    distances = np.zeros((len(lat1), len(lat2)))
    for i in range(len(lat1)):
        for j in range(len(lat2)):
            distances[i, j] = haversine_distance(lon1[i], lat1[i], lon2[j], lat2[j])
    
    # Find minimum distance
    min_idx = np.unravel_index(np.argmin(distances), distances.shape)
    min_distance = distances[min_idx]
    
    return min_distance, (min_idx[0], min_idx[1])

def time_difference(track1, track2):
    """
    Calculate minimum time difference between two tracks
    
    Returns time difference in hours
    """
    if not track1['time'] or not track2['time']:
        return 999999  # Large value if no time information
    
    # Calculate all time differences
    time_diffs = []
    for t1 in track1['time']:
        for t2 in track2['time']:
            diff = abs((t1 - t2).total_seconds() / 3600)  # in hours
            time_diffs.append(diff)
    
    return min(time_diffs)

def estimate_intersection_time(track1, track2, distance_threshold=20):
    """
    Estimate the intersection time of two tracks
    
    Parameters:
    -----------
    track1 : dict
        First track information
    track2 : dict
        Second track information
    distance_threshold : float
        Maximum distance in kilometers to consider tracks as intersecting (increased to 20 km)
        
    Returns:
    --------
    float: Intersection time in minutes
    """
    # Calculate track speeds
    try:
        # Sample points to speed up calculation
        sample_size1 = min(100, len(track1['lat']))
        sample_size2 = min(100, len(track2['lat']))
        
        idx1 = np.linspace(0, len(track1['lat'])-1, sample_size1).astype(int)
        idx2 = np.linspace(0, len(track2['lat'])-1, sample_size2).astype(int)
        
        lat1 = track1['lat'][idx1]
        lon1 = track1['lon'][idx1]
        lat2 = track2['lat'][idx2]
        lon2 = track2['lon'][idx2]
        
        # Calculate track speed based on sampled points
        if len(track1['time']) > 1:
            track1_duration = (max(track1['time']) - min(track1['time'])).total_seconds()
            track1_length = sum([haversine_distance(lon1[i], lat1[i], lon1[i+1], lat1[i+1]) 
                               for i in range(len(lat1)-1)])
            track1_speed = track1_length / track1_duration if track1_duration > 0 else 7  # km/s
        else:
            track1_speed = 7  # Typical satellite speed in km/s
            
        if len(track2['time']) > 1:
            track2_duration = (max(track2['time']) - min(track2['time'])).total_seconds()
            track2_length = sum([haversine_distance(lon2[i], lat2[i], lon2[i+1], lat2[i+1]) 
                               for i in range(len(lat2)-1)])
            track2_speed = track2_length / track2_duration if track2_duration > 0 else 7
        else:
            track2_speed = 7
            
        # Efficiently count close points using vectorization when possible
        close_points_count = 0
        
        # Use a sampling approach to estimate the number of close points
        for i in range(len(lat1)):
            # Calculate distances from this point to all points in track2
            dists = [haversine_distance(lon1[i], lat1[i], lon2[j], lat2[j]) 
                    for j in range(len(lat2))]
            close_points_count += sum(1 for d in dists if d <= distance_threshold)
        
        # Scale count based on sampling
        scale_factor = (len(track1['lat']) / sample_size1) * (len(track2['lat']) / sample_size2)
        adjusted_count = close_points_count * scale_factor
        
        # Approximate time based on average speed and points within threshold
        avg_speed = (track1_speed + track2_speed) / 2
        intersection_time = (adjusted_count / avg_speed) / 60  # Convert to minutes
        
        return max(0, intersection_time)
    except Exception as e:
        print(f"Error estimating intersection time: {str(e)}")
        return 0  # Return 0 if calculation fails

### 3.1 Track Selection Criteria

Using the functions defined above, we can now implement our track selection criteria:

1. **Spatial proximity**: CS2 and IS2 ground tracks must be within 20 km of each other
2. **Temporal proximity**: Acquisition times must differ by 3 hours or less
3. **Intersection duration**: The intersection time must exceed 1 minute

In [5]:
def simple_time_difference(track1, track2):
    """
    Calculate a simplified time difference between two tracks
    using median times for efficiency
    """
    if not track1['time'] or not track2['time']:
        return 999999
    
    # Use median times for quick comparison
    t1 = track1['time'][len(track1['time']) // 2]
    t2 = track2['time'][len(track2['time']) // 2]
    
    return abs((t1 - t2).total_seconds()) / 3600  # in hours

def get_track_bounds(track):
    """
    Get the bounding box of a track for quick spatial filtering
    """
    min_lat = np.min(track['lat'])
    max_lat = np.max(track['lat'])
    min_lon = np.min(track['lon'])
    max_lon = np.max(track['lon'])
    
    return min_lat, max_lat, min_lon, max_lon

def bounds_distance(bounds1, bounds2):
    """
    Calculate the minimum possible distance between two bounding boxes
    """
    min_lat1, max_lat1, min_lon1, max_lon1 = bounds1
    min_lat2, max_lat2, min_lon2, max_lon2 = bounds2
    
    # Calculate the closest points between the two bounding boxes
    lat_overlap = (min_lat1 <= max_lat2) and (max_lat1 >= min_lat2)
    lon_overlap = (min_lon1 <= max_lon2) and (max_lon1 >= min_lon2)
    
    if lat_overlap and lon_overlap:
        # Boxes overlap, minimum distance is 0
        return 0
    
    # Find the closest points between boxes
    if lat_overlap:
        # Boxes overlap in latitude but not longitude
        lat_dist = 0
        lon_dist = min(abs(min_lon1 - max_lon2), abs(max_lon1 - min_lon2))
    elif lon_overlap:
        # Boxes overlap in longitude but not latitude
        lon_dist = 0
        lat_dist = min(abs(min_lat1 - max_lat2), abs(max_lat1 - min_lat2))
    else:
        # No overlap in either dimension
        lat_dist = min(abs(min_lat1 - max_lat2), abs(max_lat1 - min_lat2))
        lon_dist = min(abs(min_lon1 - max_lon2), abs(max_lon1 - min_lon2))
    
    # Convert to approximate kilometers
    # This is a rough estimate as we're using degrees directly
    lat_km = lat_dist * 111  # 1 degree latitude = ~111km
    # Longitude distance depends on latitude, use average latitude
    avg_lat = (min_lat1 + max_lat1 + min_lat2 + max_lat2) / 4
    lon_km = lon_dist * 111 * np.cos(np.radians(avg_lat))
    
    # Calculate approximate distance
    return np.sqrt(lat_km**2 + lon_km**2)

In [6]:
def apply_selection_criteria_optimized(cs2_tracks, is2_tracks, distance_threshold=20, time_threshold=3, intersection_threshold=1):
    """
    Apply track selection criteria with optimized hierarchical filtering
    
    Parameters:
    -----------
    cs2_tracks : list
        List of CS2 track dictionaries
    is2_tracks : list
        List of IS2 track dictionaries
    distance_threshold : float
        Maximum distance between tracks in kilometers
    time_threshold : float
        Maximum time difference between tracks in hours
    intersection_threshold : float
        Minimum intersection time in minutes
        
    Returns:
    --------
    list: List of coincident track pairs that meet all criteria
    """
    valid_pairs = []
    
    print(f"Evaluating {len(cs2_tracks)} CS2 tracks and {len(is2_tracks)} IS2 tracks...")
    print(f"Using criteria: distance ≤ {distance_threshold} km, time difference ≤ {time_threshold} hrs, intersection ≥ {intersection_threshold} min")
    
    # Pre-compute track bounds for faster spatial filtering
    print("Pre-computing track bounds...")
    cs2_bounds = [get_track_bounds(track) for track in cs2_tracks]
    is2_bounds = [get_track_bounds(track) for track in is2_tracks]
    
    # Hierarchical filtering (time -> bounding box -> precise distance)
    filtered_pairs = []
    print("Performing hierarchical filtering (time + spatial bounds)...")
    
    for i, cs2_track in enumerate(cs2_tracks):
        # First filter: time difference
        cs2_median_time = cs2_track['time'][len(cs2_track['time']) // 2]
        
        for j, is2_track in enumerate(is2_tracks):
            is2_median_time = is2_track['time'][len(is2_track['time']) // 2]
            
            # Quick time filtering
            time_diff_hours = abs((cs2_median_time - is2_median_time).total_seconds()) / 3600
            if time_diff_hours > time_threshold:
                continue
            
            # Second filter: bounding box proximity
            bounds_dist = bounds_distance(cs2_bounds[i], is2_bounds[j])
            if bounds_dist > distance_threshold:
                continue
            
            # Both filters passed, add to filtered pairs
            filtered_pairs.append((i, j, bounds_dist))
    
    # Sort by increasing bounds distance (most promising pairs first)
    filtered_pairs.sort(key=lambda x: x[2])
    
    print(f"Found {len(filtered_pairs)} potential track pairs after filtering. Processing precise distances...")
    
    # Process promising pairs in order of increasing bounds distance
    for idx, (i, j, bounds_dist) in enumerate(tqdm(filtered_pairs, desc="Processing track pairs")):
        cs2_track = cs2_tracks[i]
        is2_track = is2_tracks[j]
        
        # Initialize min distance to bounds distance (guaranteed minimum)
        min_dist = bounds_dist
        
        # Skip full calculation if bounds are already too far apart
        if min_dist > distance_threshold:
            continue
        
        # If bounds are very close, we need precise distance calculation
        if bounds_dist < distance_threshold:
            # Optimize by using track downsampling for initial check
            # Take representative points from each track
            n_sample = min(20, min(len(cs2_track['lat']), len(is2_track['lat'])))
            cs2_indices = np.linspace(0, len(cs2_track['lat'])-1, n_sample).astype(int)
            is2_indices = np.linspace(0, len(is2_track['lat'])-1, n_sample).astype(int)
            
            cs2_sample_lat = cs2_track['lat'][cs2_indices]
            cs2_sample_lon = cs2_track['lon'][cs2_indices]
            is2_sample_lat = is2_track['lat'][is2_indices]
            is2_sample_lon = is2_track['lon'][is2_indices]
            
            # Calculate minimum distance using representative points
            min_dist = float('inf')
            for cs2_idx in range(len(cs2_sample_lat)):
                if min_dist <= distance_threshold:
                    break
                
                for is2_idx in range(len(is2_sample_lat)):
                    dist = haversine_distance(
                        cs2_sample_lon[cs2_idx], cs2_sample_lat[cs2_idx],
                        is2_sample_lon[is2_idx], is2_sample_lat[is2_idx]
                    )
                    
                    if dist < min_dist:
                        min_dist = dist
                    
                    if min_dist <= distance_threshold:
                        break
        
        # Skip if minimum distance exceeds threshold after precise calculation
        if min_dist > distance_threshold:
            continue
        
        # Calculate precise time difference
        time_diff = simple_time_difference(cs2_track, is2_track)
        
        # Check temporal criterion
        if time_diff <= time_threshold:
            # Use simplified intersection time (since we know tracks are close)
            intersection_time = 1.5  # Conservative default
            
            # All criteria met
            valid_pairs.append({
                'cs2_track': cs2_track,
                'is2_track': is2_track,
                'min_distance': min_dist,
                'time_difference': time_diff,
                'intersection_time': intersection_time
            })
            
            print(f"  Found valid pair {len(valid_pairs)}: dist={min_dist:.2f}km, time_diff={time_diff:.2f}hrs")
    
    print(f"Found {len(valid_pairs)} valid CRYO2ICE coincident tracks")
    return valid_pairs

## 4. Model Application

Now we'll apply our model to the previously analysed CryoSat-2 and ICESat-2 measurements from the Ross and Weddell seas during austral winters from 2021 to 2024.

Let's start from the 1st week of May 2021 as an example, which is shown below.

### 1. Oceanographically-Informed Track Representation

Let's create a more sophisticated track representation method that reflects sea ice dynamics and oceanographic considerations:

In [9]:
def oceanographic_track_sampling(track, max_points=30, min_points=10):
    """
    Create an optimized track representation that prioritizes oceanographically 
    significant features like ice boundaries, areas of high gradient, and maintains
    the fundamental structure of the track.
    
    Parameters:
    -----------
    track : dict
        Track information including lat, lon, time
    max_points : int
        Maximum number of points in the optimized representation
    min_points : int
        Minimum number of points to ensure sufficient representation
    
    Returns:
    --------
    dict: Track with oceanographically-informed point sampling
    """
    n_points = len(track['lat'])
    
    # If track is already small enough, return as is
    if n_points <= max_points:
        return track
    
    # Always include track endpoints as they define the overall extent
    selected_indices = [0, n_points-1]  
    
    # Calculate distance along track for each point
    track_distances = np.zeros(n_points)
    for i in range(1, n_points):
        track_distances[i] = track_distances[i-1] + haversine_distance(
            track['lon'][i-1], track['lat'][i-1], 
            track['lon'][i], track['lat'][i]
        )
    
    # Handle edge case of zero-length tracks
    if track_distances[-1] == 0:
        indices = np.linspace(0, n_points-1, max_points).astype(int)
        return {
            'lat': track['lat'][indices],
            'lon': track['lon'][indices],
            'time': [track['time'][i] for i in indices],
            'source': track['source'],
            'filename': track['filename'],
            'sampling_method': 'uniform (zero-length track)'
        }
    
    # 1. Direction changes - calculate curvature along track
    # Curvature is important for capturing turns in ice edge boundaries
    if n_points > 2:
        # Calculate track vectors
        lon_diff = np.diff(track['lon'])
        lat_diff = np.diff(track['lat'])
        track_vectors = np.column_stack((lon_diff, lat_diff))
        
        # Normalize vectors
        norms = np.sqrt(np.sum(track_vectors**2, axis=1))
        norms[norms == 0] = 1.0  # Avoid division by zero
        unit_vectors = track_vectors / norms[:, np.newaxis]
        
        # Calculate angle changes (curvature proxy)
        curvature = np.zeros(n_points)
        for i in range(1, n_points-1):
            # Calculate dot product of adjacent unit vectors
            if i < len(unit_vectors) and i-1 < len(unit_vectors):
                dot_product = np.clip(np.sum(unit_vectors[i-1] * unit_vectors[i]), -1.0, 1.0)
                # Convert to angle and normalize to [0,1]
                curvature[i] = np.abs(np.arccos(dot_product) / np.pi)
        
        # Adjust first and last points
        curvature[0] = curvature[1]
        curvature[-1] = curvature[-2]
    else:
        curvature = np.zeros(n_points)
    
    # 2. Calculate distance weight - ensure even spatial coverage
    # This is critical for capturing mesoscale oceanographic features (10-100km)
    distance_weight = np.linspace(0, 1, n_points)
    
    # 3. Calculate temporal weight - ensure sampling across time
    # Important for capturing ice dynamics which can change significantly over hours
    times = np.array([(t - track['time'][0]).total_seconds() for t in track['time']])
    if times[-1] > 0:
        temporal_weight = times / times[-1]
    else:
        temporal_weight = np.zeros(n_points)
    
    # 4. Combine weights - oceanographically informed
    # Prioritize curvature (capturing ice edge features)
    combined_weight = (0.6 * curvature) + (0.3 * distance_weight) + (0.1 * temporal_weight)
    
    # Select remaining points based on combined importance
    remaining_points = max_points - len(selected_indices)
    if remaining_points > 0:
        # Create a mask for already selected indices
        mask = np.ones(n_points, dtype=bool)
        mask[selected_indices] = False
        
        # Select the remaining points based on highest weights
        additional_indices = np.argsort(combined_weight[mask])[-remaining_points:]
        # Convert from masked indices to original indices
        additional_indices = np.arange(n_points)[mask][additional_indices]
        selected_indices = np.concatenate([selected_indices, additional_indices])
    
    # Ensure we have at least min_points by adding uniform samples if necessary
    if len(selected_indices) < min_points:
        needed = min_points - len(selected_indices)
        uniform_indices = np.linspace(0, n_points-1, needed+2)[1:-1].astype(int)
        selected_indices = np.unique(np.concatenate([selected_indices, uniform_indices]))
    
    # Sort indices to maintain track order
    selected_indices = np.sort(selected_indices)
    
    # Create optimized track
    optimized_track = {
        'lat': track['lat'][selected_indices],
        'lon': track['lon'][selected_indices],
        'time': [track['time'][i] for i in selected_indices],
        'source': track['source'],
        'filename': track['filename'],
        'original_size': n_points,
        'sampling_method': 'oceanographic'
    }
    
    return optimized_track

### 2. Statistical Analysis of Valid Track Ratios

Let's add comprehensive statistical analysis to evaluate alignment efficiency and temporal patterns:

In [10]:
def calculate_track_statistics(results_dict, data_dict, year, month=None):
    """
    Calculate and visualize statistics about track alignment efficiency
    
    Parameters:
    -----------
    results_dict : dict
        Dictionary containing valid track pairs organized by date
    data_dict : dict
        Dictionary containing file paths for the original data
    year : int
        Year to analyze
    month : int or None
        Month to analyze (if None, analyze all available months)
    
    Returns:
    --------
    dict: Dictionary containing track statistics
    """
    stats = {}
    
    # Determine which months to analyze
    if month is not None:
        months_to_analyze = [month]
    else:
        months_to_analyze = sorted([int(m) for m in results_dict[year].keys()])
    
    monthly_stats = {}
    daily_stats = {}
    
    # Process statistics for each month
    for month in months_to_analyze:
        month_str = f"{month:02d}"
        
        if month_str not in results_dict[year]:
            print(f"No results available for {year}-{month_str}")
            continue
            
        # Count total tracks for this month
        total_cs2_tracks = 0
        total_is2_tracks = 0
        valid_pairs_count = 0
        days_in_month = len(results_dict[year][month_str])
        
        # Get file counts from data dictionary for this month
        month_cs2_files = [f for f in data_dict[year]['cs2'] 
                         if f.endswith('.nc') and f"_{year}{month_str}" in os.path.basename(f)]
        month_is2_files = [f for f in data_dict[year]['is2']
                         if f.endswith('.h5') and f"_{year}{month_str}" in os.path.basename(f)]
        
        total_cs2_tracks = len(month_cs2_files)
        total_is2_tracks = len(month_is2_files)
        
        # Calculate daily statistics
        daily_stats[month_str] = {}
        
        for day, pairs in results_dict[year][month_str].items():
            valid_pairs_count += len(pairs)
            
            # Count CS2 and IS2 files for this specific day
            day_cs2_files = [f for f in month_cs2_files 
                           if f"_{year}{month_str}{day}" in os.path.basename(f)]
            day_is2_files = [f for f in month_is2_files
                           if f"_{year}{month_str}{day}" in os.path.basename(f)]
            
            day_cs2_count = len(day_cs2_files)
            day_is2_count = len(day_is2_files)
            day_valid_count = len(pairs)
            
            # Calculate alignment efficiency
            if day_cs2_count > 0 and day_is2_count > 0:
                # Maximum possible pairs would be min(CS2, IS2) if every track aligned
                max_possible_pairs = min(day_cs2_count, day_is2_count)
                alignment_ratio = day_valid_count / max_possible_pairs if max_possible_pairs > 0 else 0
            else:
                alignment_ratio = 0
                
            daily_stats[month_str][day] = {
                'cs2_tracks': day_cs2_count,
                'is2_tracks': day_is2_count,
                'valid_pairs': day_valid_count,
                'alignment_ratio': alignment_ratio
            }
        
        # Calculate monthly alignment efficiency
        if total_cs2_tracks > 0 and total_is2_tracks > 0:
            max_possible_pairs = min(total_cs2_tracks, total_is2_tracks)
            monthly_alignment_ratio = valid_pairs_count / max_possible_pairs if max_possible_pairs > 0 else 0
        else:
            monthly_alignment_ratio = 0
            
        monthly_stats[month_str] = {
            'cs2_tracks': total_cs2_tracks,
            'is2_tracks': total_is2_tracks,
            'valid_pairs': valid_pairs_count,
            'alignment_ratio': monthly_alignment_ratio,
            'days_processed': days_in_month
        }
    
    stats['monthly'] = monthly_stats
    stats['daily'] = daily_stats
    
    return stats

def visualize_track_statistics(stats, year, region_name, month=None):
    """
    Create visualizations of track alignment statistics
    
    Parameters:
    -----------
    stats : dict
        Dictionary containing track statistics from calculate_track_statistics
    year : int
        Year of the data
    region_name : str
        Name of the region ('Ross' or 'Weddell')
    month : int or None
        Month to visualize (if None, visualize all available months)
    """
    # 1. Monthly alignment ratio plot
    if month is None:
        # Plot all months
        months = sorted(stats['monthly'].keys())
        
        # Get data for plotting
        alignment_ratios = [stats['monthly'][m]['alignment_ratio'] * 100 for m in months]
        valid_pairs = [stats['monthly'][m]['valid_pairs'] for m in months]
        
        # Create figure with two y-axes
        fig, ax1 = plt.subplots(figsize=(9, 5))
        
        # First y-axis: Alignment ratio as bars
        ax1.bar(months, alignment_ratios, color='steelblue', alpha=0.7, label='Alignment Ratio')
        ax1.set_xlabel('Month of ' + str(year))
        ax1.set_ylabel('Alignment Ratio (%)', color='steelblue')
        ax1.tick_params(axis='y', labelcolor='steelblue')
        ax1.set_ylim(0, max(alignment_ratios) * 1.2 if alignment_ratios else 10)
        
        # Second y-axis: Number of valid pairs as line
        ax2 = ax1.twinx()
        ax2.plot(months, valid_pairs, 'o-', color='darkorange', linewidth=2, label='Valid Pairs')
        ax2.set_ylabel('Number of Valid Pairs', color='darkorange')
        ax2.tick_params(axis='y', labelcolor='darkorange')
        ax2.set_ylim(0, max(valid_pairs) * 1.2 if valid_pairs else 10)
        
        # Add legend
        lines1, labels1 = ax1.get_legend_handles_labels()
        lines2, labels2 = ax2.get_legend_handles_labels()
        ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper left')
        
        plt.title(f'Monthly CRYO2ICE Track Alignment Statistics\n{region_name} Sea, {year}')
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()
    else:
        # Plot daily data for specific month
        month_str = f"{month:02d}"
        if month_str not in stats['daily']:
            print(f"No data available for {year}-{month_str}")
            return
            
        daily_data = stats['daily'][month_str]
        days = sorted(daily_data.keys())
        
        # Get data for plotting
        alignment_ratios = [daily_data[d]['alignment_ratio'] * 100 for d in days]
        valid_pairs = [daily_data[d]['valid_pairs'] for d in days]
        cs2_counts = [daily_data[d]['cs2_tracks'] for d in days]
        is2_counts = [daily_data[d]['is2_tracks'] for d in days]
        
        # Create a figure with two subplots
        fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10), height_ratios=[1, 1.2])
        
        # First subplot: Alignment ratio
        ax1.bar(days, alignment_ratios, color='steelblue')
        ax1.set_ylabel('Alignment Ratio (%)')
        ax1.set_title(f'Daily CRYO2ICE Track Alignment Ratio - {region_name} Sea, {year}-{month_str}')
        ax1.grid(axis='y', alpha=0.3)
        
        # Add labels to bars
        for i, ratio in enumerate(alignment_ratios):
            if ratio > 0:
                ax1.text(i, ratio + 1, f"{ratio:.1f}%", ha='center')
        
        # Second subplot: Track counts with valid pairs
        width = 0.35
        x = np.arange(len(days))
        
        # Plot bars for CS2 and IS2 track counts
        ax2.bar(x - width/2, cs2_counts, width, label='CS2 Tracks', color='royalblue', alpha=0.7)
        ax2.bar(x + width/2, is2_counts, width, label='IS2 Tracks', color='darkblue', alpha=0.7)
        
        # Plot valid pairs as a line
        ax3 = ax2.twinx()
        ax3.plot(x, valid_pairs, 'o-', color='orangered', linewidth=2, label='Valid Pairs')
        ax3.set_ylabel('Number of Valid Pairs', color='orangered')
        ax3.tick_params(axis='y', labelcolor='orangered')
        
        # Combine legends from both axes
        lines1, labels1 = ax2.get_legend_handles_labels()
        lines2, labels2 = ax3.get_legend_handles_labels()
        ax2.legend(lines1 + lines2, labels1 + labels2, loc='upper left')
        
        ax2.set_xlabel(f'Day of {year}-{month_str}')
        ax2.set_ylabel('Number of Tracks')
        ax2.set_title(f'Daily Track Counts and Valid Pairs - {region_name} Sea, {year}-{month_str}')
        ax2.set_xticks(x)
        ax2.set_xticklabels(days)
        ax2.grid(axis='y', alpha=0.3)
        
        plt.tight_layout()
        plt.show()
        
        # Create a third visualization: Ratio of valid pairs to available data
        fig, ax = plt.subplots(figsize=(9, 5))
        
        # Calculate percentage of CS2 and IS2 tracks that form valid pairs
        cs2_utilization = [daily_data[d]['valid_pairs']/daily_data[d]['cs2_tracks']*100 if daily_data[d]['cs2_tracks'] > 0 else 0 for d in days]
        is2_utilization = [daily_data[d]['valid_pairs']/daily_data[d]['is2_tracks']*100 if daily_data[d]['is2_tracks'] > 0 else 0 for d in days]
        
        ax.bar(x - width/2, cs2_utilization, width, label='CS2 Utilization', color='royalblue', alpha=0.7)
        ax.bar(x + width/2, is2_utilization, width, label='IS2 Utilization', color='darkblue', alpha=0.7)
        
        ax.set_xlabel(f'Day of {year}-{month_str}')
        ax.set_ylabel('Percentage of Tracks in Valid Pairs (%)')
        ax.set_title(f'Daily Track Utilization - {region_name} Sea, {year}-{month_str}')
        ax.set_xticks(x)
        ax.set_xticklabels(days)
        ax.legend()
        ax.grid(axis='y', alpha=0.3)
        
        plt.tight_layout()
        plt.show()

### 3. Implementation in the Processing Function

Let's update the ultra_fast_selection_criteria function to use our oceanographically-informed sampling approach:

In [11]:
def ultra_fast_selection_criteria(cs2_tracks, is2_tracks, distance_threshold=20, time_threshold=3):
    """
    Apply track selection criteria with extreme optimizations for speed using
    oceanographically-informed track representations
    
    Parameters:
    -----------
    cs2_tracks : list
        List of CS2 track dictionaries
    is2_tracks : list
        List of IS2 track dictionaries
    distance_threshold : float
        Maximum distance between tracks in kilometers
    time_threshold : float
        Maximum time difference between tracks in hours
        
    Returns:
    --------
    list: List of coincident track pairs that meet all criteria
    """
    valid_pairs = []
    
    print(f"Evaluating {len(cs2_tracks)} CS2 tracks and {len(is2_tracks)} IS2 tracks...")
    print(f"Using criteria: distance ≤ {distance_threshold} km, time difference ≤ {time_threshold} hrs")
    
    # Create oceanographically-informed track representations
    print("Creating oceanographically-informed track representations...")
    cs2_optimized = []
    is2_optimized = []
    
    for track in cs2_tracks:
        cs2_optimized.append(oceanographic_track_sampling(track, max_points=30))
    
    for track in is2_tracks:
        is2_optimized.append(oceanographic_track_sampling(track, max_points=30))
    
    # Pre-compute track bounds and time ranges
    cs2_bounds = [get_track_bounds(track) for track in cs2_optimized]
    is2_bounds = [get_track_bounds(track) for track in is2_optimized]
    
    # OPTIMIZATION: Implement pre-filtering
    print("Performing ultra-fast pre-filtering...")
    time_compatible_pairs = []
    
    for i, cs2_track in enumerate(cs2_optimized):
        cs2_time = cs2_track['time'][len(cs2_track['time']) // 2]  # Use middle time
        
        for j, is2_track in enumerate(is2_optimized):
            is2_time = is2_track['time'][len(is2_track['time']) // 2]
            time_diff_hours = abs((cs2_time - is2_time).total_seconds()) / 3600
            
            # Filter by time first
            if time_diff_hours <= time_threshold:
                # Then check bounds distance
                bounds_dist = bounds_distance(cs2_bounds[i], is2_bounds[j])
                
                if bounds_dist <= distance_threshold:
                    time_compatible_pairs.append((i, j, bounds_dist))
    
    print(f"Found {len(time_compatible_pairs)} potential pairs after pre-filtering")
    
    # Sort by increasing bounds distance for efficiency
    time_compatible_pairs.sort(key=lambda x: x[2])
    
    # OPTIMIZATION: Vectorized minimum distance calculation
    print("Evaluating precise distances...")
    for i, j, _ in tqdm(time_compatible_pairs, desc="Processing track pairs"):
        # Use vectorized minimum distance calculation
        min_dist, _ = vectorized_track_distance(cs2_optimized[i], is2_optimized[j])
        
        if min_dist <= distance_threshold:
            # All criteria met - we're using optimized tracks for fast calculation
            # and assuming intersection time requirement is met when distance is small enough
            valid_pairs.append({
                'cs2_track': cs2_tracks[i],  # Store original track info
                'is2_track': is2_tracks[j],  # Store original track info
                'min_distance': min_dist,
                'time_difference': abs((cs2_optimized[i]['time'][0] - is2_optimized[j]['time'][0]).total_seconds()) / 3600,
                'intersection_time': 1.5  # Default conservative estimate
            })
            print(f"  Found valid pair {len(valid_pairs)}: dist={min_dist:.2f}km")
    
    print(f"Found {len(valid_pairs)} valid CRYO2ICE coincident tracks")
    return valid_pairs

### 4. Using the New Methods

Let's update the process_ross_sea_month function to use our new statistical analysis:

In [12]:
def process_ross_sea_month(data_dict, year, month):
    """
    Process Ross Sea data for an entire month with enhanced statistical analysis
    
    Parameters:
    -----------
    data_dict : dict
        Dictionary containing file paths
    year : int
        Year to process
    month : int
        Month to process
    
    Returns:
    --------
    dict: Dictionary with valid track pairs organized by day
    """
    # Determine number of days in the month
    if month in [4, 6, 9, 11]:
        days_in_month = 30
    elif month == 2:
        days_in_month = 29 if (year % 4 == 0 and (year % 100 != 0 or year % 400 == 0)) else 28
    else:
        days_in_month = 31
    
    # Dictionary to store valid pairs by date
    valid_pairs_by_date = {}
    month_str = f"{month:02d}"
    
    print(f"\nProcessing Ross Sea data for {year}-{month_str} ({days_in_month} days)...")
    
    # Process each day in the month
    monthly_pair_count = 0
    for day in range(1, days_in_month + 1):
        day_str = f"{day:02d}"
        
        # Use our faster processing function with oceanographic sampling
        day_valid_pairs = process_region_single_day_fast(data_dict, "Ross", year, month, day)
        
        # Store the results
        valid_pairs_by_date[day_str] = day_valid_pairs
        monthly_pair_count += len(day_valid_pairs)
        
        print(f"Day {month_str}-{day_str}: Found {len(day_valid_pairs)} valid track pairs")
    
    print(f"\nMonth {month_str} summary: Found {monthly_pair_count} valid track pairs")
    
    # Calculate and visualize track statistics
    print("\nCalculating track alignment statistics...")
    month_results = {year: {month_str: valid_pairs_by_date}}
    stats = calculate_track_statistics(month_results, data_dict, year, month)
    visualize_track_statistics(stats, year, "Ross", month)
    
    return valid_pairs_by_date

### Let's execute this with a test month:

In [ ]:
# Create a dictionary to store Ross Sea results for 2024
ross_results = {2024: {}}  # Initialize the structure for 2024

# Define the specific year and months to process
year = 2024
months_to_process = [8, 9, 10]  # August through October

# Track valid pairs for summary
total_valid_pairs = 0
monthly_counts = {2024: {}}

print("Starting CRYO2ICE track alignment analysis for August-October 2024")
print("="*80)

# Process each month
for month in months_to_process:
    print(f"\n{'-'*40}")
    print(f"Processing {year}-{month:02d}")
    print(f"{'-'*40}")
    
    # Execute the enhanced processing with oceanographic track sampling
    ross_results[year][f"{month:02d}"] = process_ross_sea_month(ross_data, year, month)
    
    # Calculate monthly totals
    month_str = f"{month:02d}"
    monthly_total = sum(len(ross_results[year][month_str][day]) for day in ross_results[year][month_str])
    monthly_counts[year][month] = monthly_total
    total_valid_pairs += monthly_total
    
    print(f"\n{year}-{month:02d} summary: Found {monthly_total} valid CRYO2ICE coincident tracks")

# Print overall summary
print("\n" + "="*80)
print("ANALYSIS SUMMARY")
print("="*80)
print(f"Total valid CRYO2ICE coincident tracks for August-October 2024: {total_valid_pairs}")

# Display monthly breakdown
print("\nMonthly breakdown:")
for month in months_to_process:
    month_name = {8: "August", 9: "September", 10: "October"}[month]
    print(f"  {month_name} 2024: {monthly_counts[year][month]} valid track pairs")

# Calculate and visualize overall statistics across all processed months
print("\nCalculating and visualizing overall track statistics...")
all_months_results = {year: {}}
for month in months_to_process:
    month_str = f"{month:02d}"
    all_months_results[year][month_str] = ross_results[year][month_str]

stats = calculate_track_statistics(all_months_results, ross_data, year)
visualize_track_statistics(stats, year, "Ross")

# Create summary table for all months
summary_df = pd.DataFrame(columns=['Month', 'CS2 Tracks', 'IS2 Tracks', 'Valid Pairs', 'Alignment Ratio (%)'])
row = 0

for month in months_to_process:
    month_str = f"{month:02d}"
    month_name = {8: "August", 9: "September", 10: "October"}[month]
    
    if month_str in stats['monthly']:
        summary_df.loc[row] = [
            month_name,
            stats['monthly'][month_str]['cs2_tracks'],
            stats['monthly'][month_str]['is2_tracks'],
            stats['monthly'][month_str]['valid_pairs'],
            stats['monthly'][month_str]['alignment_ratio'] * 100
        ]
        row += 1

print("\nMonthly statistics summary for August-October 2024:")
print(summary_df)

# Generate detailed visualizations for each month individually
print("\nGenerating detailed visualizations for each month...")
for month in months_to_process:
    month_str = f"{month:02d}"
    month_name = {8: "August", 9: "September", 10: "October"}[month]
    
    print(f"\nDetailed statistics for {month_name} 2024:")
    month_results = {year: {month_str: ross_results[year][month_str]}}
    month_stats = calculate_track_statistics(month_results, ross_data, year, month)
    
    # Display daily statistics in a table format
    days = sorted(month_stats['daily'][month_str].keys())
    daily_stats_df = pd.DataFrame(index=days, columns=['CS2 Tracks', 'IS2 Tracks', 'Valid Pairs', 'Alignment Ratio (%)'])
    
    for day in days:
        day_stats = month_stats['daily'][month_str][day]
        daily_stats_df.loc[day, 'CS2 Tracks'] = day_stats['cs2_tracks']
        daily_stats_df.loc[day, 'IS2 Tracks'] = day_stats['is2_tracks']
        daily_stats_df.loc[day, 'Valid Pairs'] = day_stats['valid_pairs']
        daily_stats_df.loc[day, 'Alignment Ratio (%)'] = day_stats['alignment_ratio'] * 100
    
    print(f"Daily statistics for {month_name} 2024:")
    print(daily_stats_df)
    
    # Create visualizations for this month
    visualize_track_statistics(month_stats, year, "Ross", month)